1. See the difference in graph representation. (Import graph) (graph attributes: https://github.com/mggg-states/PA-shapefiles#metadata)

2. Fix the PopulatedGraph. How is the population info extracted in the algorithm?

3. Define updaters? (To save the value of some functions like total pop in districts) What is it used for? (from gerrychain.updaters import Tally, cut_edges)

4. Generate initial solution with Gerrychain. 

    - Every district must contain at least one candidate. DONE.

5. Use optimization


Metrics directory?

Region surchargery? 

Spanning tree is none in bipartition_tree for the initial solution

How do you call it sampling method if you generate the initial solution with random_spanning_tree?

!!! 

- Havaalaninin districtinde hic center yok, nasil olur? Ayrica o kadar buyuk bir bolgede bir center olmali.

- Yeni plotta bir district eksik???


In [ ]:
from tree import recursive_tree_part, bipartition_tree
from data_utils import DataHandler
from functools import partial

In [ ]:
# Load graph
handler = DataHandler()

graph = handler.load_ggraph()
chicago = handler.load_chicago()
candidates = handler.load_candidates()  # set of nodes
geo_centers = handler.load_geo_centers()
# Load travel times
#travel_times = load(file_path='/Users/kirtisoglu/Documents/GitHub/Allocation-of-Primary-Care-Centers-in-Chicago/prepared_data/travel_times.pkl.gz')

In [ ]:
pop = sum(graph.nodes[node]['pop'] for node in graph.nodes)
pop

In [ ]:
parts = list(range(100))
pop_target = pop / 100 

initial = recursive_tree_part(graph, parts, pop_target, pop_col='pop', epsilon= 0.1, node_repeats = 1, method = partial(bipartition_tree, max_attempts=10000))
    

In [ ]:
for node, district_id in initial.items():
    graph.nodes[node]['district'] = district_id

In [ ]:
import updaters 
from partition import Partition, GeographicPartition


my_updaters = {
    "population": updaters.Tally('pop', alias="population"),
    "cut_edges": updaters.cut_edges,
    "perimeter": updaters.perimeter,
    "area": updaters.Tally("area", alias="area"),
}

initial_partition = GeographicPartition(
    graph,
    assignment='district',
    updaters=my_updaters
)

In [ ]:
import random
node = random.choice(list(graph.nodes))
graph.nodes[node]

In [ ]:
import matplotlib.pyplot as plt
initial_partition.plot(geometries=chicago)

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.set_yticks([])
ax.set_xticks([])
ax.set_title("Initial Partition in MN")
initial_partition.plot(ax=ax, cmap='tab20c', geometries=chicago)
# groceries.plot(ax=base, marker='o', color='red', markersize=5) in case groceries is a geodataframe.

In [ ]:
from chain import MarkovChain
from proposals import recom

# ...define constraints, accept, partition, total_steps here...

proposal = partial(recom, pop_col='pop', pop_target=pop_target, epsilon=.01, node_repeats=1, method = partial(
        bipartition_tree,
        max_attempts=100,
        allow_pair_reselection=True
    ))


In [ ]:
from constraints import contiguous
import accept


recom_chain = MarkovChain(
    proposal=proposal,
    constraints=[contiguous],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=40
)

In [ ]:
assignment_list = []
partitions = {}

for i, item in enumerate(recom_chain):
    print(f"Finished step {i+1}/{len(recom_chain)}", end="\r")
    assignment_list.append(item.assignment)
    partitions[i] = item

In [ ]:
"""import pandas as pd


district_data = []

for i, partition in enumerate(recom_chain):
    for district_name in partition["population"].keys():
        population = partition["population"][district_name]
        #area = partition["area"][district_name]
        n_cut_edges = len(partition["cut_edges"])
        district_data.append((i, district_name, population, n_cut_edges))

df = pd.DataFrame(
    district_data,
    columns=[
        'step',
        'district_name',
        'population',
        'n_cut_edges'
    ]
)"""

In [ ]:
#df[df['step'] == 11]

In [ ]:

nodes = [node for node in graph.nodes if graph.nodes[node]['district']==21 or graph.nodes[node]['district']==62]
subgraph = graph.subgraph(nodes)
subpop = sum(graph.nodes[node]['pop'] for node in subgraph.nodes)
subpop

In [ ]:
chicago['district'] = [graph.nodes[node]['district'] for node in chicago.index]

In [ ]:
"""subparts = list(range(2))
subpop_target = subpop / 2

subinitial = recursive_tree_part(subgraph, subparts, subpop_target, pop_col='pop', epsilon= 0.1, node_repeats = 1, method = partial(bipartition_tree, max_attempts=10000))


sub_initial_partition = GeographicPartition(
    subgraph,
    assignment='district',
    updaters=my_updaters
)"""

In [ ]:
"""subgraph_nodes = list(subgraph.nodes())
subchicago = chicago.loc[subgraph_nodes]

sub_initial_partition.plot(geometries=subchicago)

# Use this:
#chicago.loc[np.random.choice(chicago.index, 30), 'POP2010'] = np.nan
#chicago.plot(column='POP2010');
# or"""

In [ ]:
frozen_districts = initial_partition.parts
districts = {}
candidates = {}
centers = set()

for key, frozenset in frozen_districts.items():
    districts[key] = set(frozenset)
    candidates[key] = []
    for node in districts[key]:
        if graph.nodes[node]['real_phc']==True:
                candidates[key].append(node)
                centers.add(node)

print("Candidates:", candidates)
#print("Centers:", centers)

In [ ]:
"""import plotly.express as px
import pandas as pd

def plot(data, centers, attribute: str):
    # Ensure data[attribute] has appropriate type for indexing
    #data[attribute] = pd.Categorical(data[attribute])
    
    # Map each cluster to a color using a cycle of the Plotly qualitative palette
    #colors = px.colors.qualitative.Plotly  # This is an example palette
    #color_map = {cat: colors[i % len(colors)] for i, cat in enumerate(data[attribute].cat.categories)}
    #data['color'] = data[attribute].map(color_map)

    fig = px.choropleth_mapbox(
        data,
        geojson=data.geometry.__geo_interface__,
        locations=data.index,
        color='district',
        mapbox_style="open-street-map",
        center={"lat": data.geometry.centroid.y.mean(), "lon": data.geometry.centroid.x.mean()},
        height=800,
        zoom=10,
        opacity=0.5,
        color_discrete_map="identity",  # Ensure this uses the direct mapping of assigned colors
        hover_data=[data['pop']]  # Show population data on hover
    )

    # Add cluster centers as markers
    for center in centers:
        center_point = data.loc[center].geometry.centroid
        fig.add_scattermapbox(
            lat=[center_point.y],
            lon=[center_point.x],
            mode='markers',
            marker=dict(size=8, color='black'),  # Black markers for centers
            name=f'Candidate={center}'
        )

    return fig.show()

chicago['district'] = [graph.nodes[node]['district'] for node in chicago.index]
plot(chicago, centers, attribute='district')"""

In [ ]:
regions = chicago.dissolve(by='district', as_index=False)


In [37]:
import folium
import folium.plugins 


# m = folium.Map([41.85, -87.68], zoom_start=10)
m = regions.explore(
    column="district",  # make choropleth based on "BoroName" column
    tooltip="district",  # show "district" value in tooltip (on hover)
    popup=True,  # show all values in popup (on click)
    tiles="OpenStreetMap",  # use "CartoDB positron" or "OpenStreetMap" tiles
    cmap="Set1",  # use "Set1" matplotlib colormap
    style_kwds=dict(color="black"),  # use black outline
    legend_kwds=dict(colorbar=False),
    #tooltip_kwds=dict(labels=False),  # do not show column label in the tooltip
    #smooth_factor=2,
    #fill_opacity=0.3,  #  transparency of fill colors
    #line_opacity=0.1,  # to de-emphasize border lines
    #fill_color="RdYlGn_r",  # or "YlGn"
    #nan_fill_color="white", # Also see nan_fill_opacity=0.4,
    highlight=True,
    name = "chicago"
)

#Adds a button to enable/disable zoom scrolling
folium.plugins.ScrollZoomToggler().add_to(m)

# To make the map full screen
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)


geo_centers.explore(
    m=m,  # pass the map object
    color="black",  # use red color on all points
    marker_kwds=dict(radius=3, fill=True),  # make marker radius 10px with fill
    name="Candidates",  # name of the layer in the map
)
#folium.TileLayer("CartoDB positron", show=False).add_to(m)  
# use folium to add alternative tiles
folium.LayerControl().add_to(m)  # use folium to add layer control




# Side by side Layers: control=False  to add a layer control to your map
#m = folium.Map(location=(30, 20), zoom_start=4)

#layer_right = folium.TileLayer('openstreetmap')
#layer_left = folium.TileLayer('cartodbpositron')

#sbs = folium.plugins.SideBySideLayers(layer_left=layer_left, layer_right=layer_right)

#layer_left.add_to(m)
#layer_right.add_to(m)
#sbs.add_to(m)

m

In [ ]:
"""folium.GeoJson(
    gdf,
    name="Subway Stations",
    marker=folium.Circle(radius=4, fill_color="orange", fill_opacity=0.4, color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line", "notes"]),
    popup=folium.GeoJsonPopup(fields=["name", "line", "href", "notes"]),
    style_function=lambda x: {
        "fillColor": colors[x['properties']['service_level']],
        "radius": (x['properties']['lines_served'])*30,
    },
    highlight_function=lambda x: {"fillOpacity": 0.8},
    zoom_on_click=True,
).add_to(m)"""